using https://splines.readthedocs.io/

In [ ]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import splines

In [ ]:
from unit_quaternion import Quaternion, canonicalize_quaternion_sequence
from helper import angles2quat, prepare_axis, update_plot

In [ ]:
rotations = [
    angles2quat(0, 0, 0),
    angles2quat(45, 0, 0),
    angles2quat(90, 45, 0),
    angles2quat(90, 90, 0),
    angles2quat(91, 91, 0),
    angles2quat(180, 0, 90),
]

`alpha=0.5`: centripetal

In [ ]:
assert rotations == list(canonicalize_quaternion_sequence(rotations))

In [ ]:
vertices = [q.xyzw for q in rotations]
s = splines.CatmullRom(vertices, alpha=0.5, endconditions='closed')

In [ ]:
grid = [0, 2, None, None, None, 3, 6]

In [ ]:
s = splines.ReParameterizedCurve(s, grid)

In [ ]:
s.grid

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
interpolated = s.evaluate(times)

In [ ]:
normalized = [Quaternion(w, (x, y, z)).normalize() for x, y, z, w in interpolated]

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(1, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, [rot])

In [ ]:
ani = FuncAnimation(fig, ani_func, frames=normalized, interval=60)
display(HTML(ani.to_jshtml(default_mode='loop')))

In [ ]:
from scipy.spatial.transform import Rotation, RotationSpline

In [ ]:
s2 = RotationSpline(s.grid, Rotation([rot.xyzw for rot in (rotations + rotations[:1])]))

In [ ]:
s2.times

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection='dumb3d'))
collections = prepare_axis(2, ax=ax)
plt.close(fig)

In [ ]:
def ani_func(rot):
    return update_plot(collections, rot)

In [ ]:
ani = FuncAnimation(
    fig,
    ani_func,
    init_func=lambda: None,
    frames=zip(normalized, s2(times).as_quat()),
    interval=60,
)
display(HTML(ani.to_jshtml(default_mode='loop')))